In [61]:
import pandas as pd

## TO DO -- optinally parameterize restock volume, and posting stack size (for multiple stack sizes do we just send in multiple records?)

In [7]:
class EXPORT():
    def __init__(self, df):
        self.validate_and_run(df)        

    def auction_operation(self, name, min_price, normal_price, max_price, stack_size=5, **kwargs):
        # singles (sold at 1c higher to order properly behind stack)
        s  =   (f's{name}^T^SaboveMax^SmaxPrice^SnormalPrice^S{normal_price+1}'+
                f'^Sduration^N3^SmatchStackSize^b^SkeepQuantity^N0^SpostCap^N5'+
                f'^SbidPercent^N1^SstackSizeIsCap^B^SmaxPrice^S{max_price+1}'+
                f'^SignoreLowDuration^N0^SstackSize^N1^SkeepPosted^N0^Sundercut^S1c'+
                f'^SmaxExpires^N0^SpriceReset^SminPrice^ScancelRepostThreshold^S1g'+
                f'^ScancelRepost^B^ScancelUndercut^B^SminPrice^S{min_price+1}'+
                f'^SkeepQtySources^T^t^Sblacklist'
               ) 
        # stacks
        s += '^S^t^S'
        s +=   (f'S{name}^T^SaboveMax^SmaxPrice^SnormalPrice^S{normal_price}'+
                f'^Sduration^N3^SmatchStackSize^b^SkeepQuantity^N0^SpostCap^N20'+
                f'^SbidPercent^N1^SstackSizeIsCap^B^SmaxPrice^S{max_price}'+
                f'^SignoreLowDuration^N0^SstackSize^N{stack_size}^SkeepPosted^N0^Sundercut^S1c'+
                f'^SmaxExpires^N0^SpriceReset^SminPrice^ScancelRepostThreshold^S1g'+
                f'^ScancelRepost^B^ScancelUndercut^B^SminPrice^S{min_price}'+
                f'^SkeepQtySources^T^t^Sblacklist'
               )
        return s


    def shopping_operation(self, name, buy_price, restock=20, **kwargs):
        return (f'{name}^T^SevenStacks^b^SshowAboveMaxPrice^b^SmaxPrice^S{buy_price}'+
                f'^SrestockQuantity^N{restock}^SrestockSources^T^Salts^b^Sauctions^B^Sguild^b^Sbank^B'
               )

    def group_operation_map(self, name, **kwargs):
        return (f'{name}^T^SMailing^T^N1^S#Default^t^SAuctioning^T^N1^Ss{name}^N2^SS{name}'+
                f'^Soverride^B^t^SCrafting^T^N1^S#Default^t^SWarehousing^T^N1'+
                f'^S#Default^t^SVendoring^T^N1^S#Default^t^SShopping^T^N1^S{name}'+
                f'^t^SSniper^T^N1^S#Default'
               )

    def group(self, name, item_id, **kwargs):
        return f'group:{name},i:{item_id}'

    # MULTI ITEM FUNCTIONS
    
    def dispatcher(self, fn, delim, item_list):
        return delim.join([fn(**item) for item in item_list])

    def multi_auction_operation(self, item_list):
        return self.dispatcher(self.auction_operation, '^S^t^S', item_list)

    def multi_shopping_operation(self, item_list):
        return self.dispatcher(self.shopping_operation, '^t^t^S', item_list)

    def multi_group_operation_map(self, item_list):
        return self.dispatcher(self.group_operation_map, '^t^t^S', item_list)

    def multi_group(self, item_list):
        return self.dispatcher(self.group, ',', item_list)
    
    def validate_and_run(self, df):
        if 'name' not in df.columns:         
            print('! Error - missing name column')        
            return
        if 'item_id' not in df.columns:      
            print('! Error - missing item_id column')
            return
        if 'buy_price' not in df.columns:    
            print('! Error - missing buy_price column')
            return
        if 'min_price' not in df.columns:    
            print('! Error - missing min_price column')
            return
        if 'normal_price' not in df.columns: 
            print('! Error - normal_price name column')
            return
        if 'max_price' not in df.columns:    
            print('! Error - max_price name column')
            return
        
        if not df[['min_price','max_price','buy_price','normal_price']].applymap(lambda x: x%1==0).all().all():
            print('! Error - price values contain non-ints')
            return
        
        # Validation passed, generate export
        data = df.to_dict('records') 
        
        print(f'^1^T^S'+
         f'operations^T^S'+
             f'Mailing^T^S#Default^T^Srestock^b^SkeepQty^N0^SmaxQtyEnabled^b^Starget^S^SrestockSources^T^Sguild^b^Sbank^b^t^SmaxQty^N10^t^t^S'+
             f'Auctioning^T^S{self.multi_auction_operation(data)}^S^t^t^S'+
             f'Crafting^T^S#Default^T^SminRestock^N1^SminProfit^S100g^ScraftPriceMethod^S^SmaxRestock^N3^t^t^S'+
             f'Warehousing^T^S#Default^T^SstackSize^N0^SmoveQuantity^N0^SkeepBankQuantity^N0^SrestockKeepBankQuantity^N0^SrestockQuantity^N0^SrestockStackSize^N0^SkeepBagQuantity^N0^t^t^S'+
             f'Vendoring^T^S#Default^T^SvsMarketValue^Sdbmarket^SenableBuy^B^SvsMaxMarketValue^S0c^SsellAfterExpired^N20^SvsDestroyValue^Sdestroy^SrestockQty^N0^SkeepQty^N0^SvsMaxDestroyValue^S0c^SrestockSources^T^Salts_ah^b^Sah^b^Sguild^b^Salts^b^Smail^b^Sbank^b^t^SsellSoulbound^b^SenableSell^B^t^t^S'+
             f'Shopping^T^S{self.multi_shopping_operation(data)}^t^t^t^S'+
             f'Sniper^T^S#Default^T^SbelowPrice^Smax(vendorsell,~`ifgt(DBRegionMarketAvg,~`250000g,~`0.8,~`ifgt(DBRegionMarketAvg,~`100000g,~`0.7,~`ifgt(DBRegionMarketAvg,~`50000g,~`0.6,~`ifgt(DBRegionMarketAvg,~`25000g,~`0.5,~`ifgt(DBRegionMarketAvg,~`10000g,~`0.4,~`ifgt(DBRegionMarketAvg,~`5000g,~`0.3,~`ifgt(DBRegionMarketAvg,~`2000g,~`0.2,~`ifgt(DBRegionMarketAvg,~`1000g,~`0.1,~`0.05))))))))~`*~`DBRegionMarketAvg)^t^t^t^S'+
         f'groupOperations^T^S{self.multi_group_operation_map(data)}^t^t^t^S'+
         f'groupExport^S{self.multi_group(data)}^t^^'
        )
        
        

In [8]:
def example_export_input():
    return pd.DataFrame([
            {'name':'Wildvine',
             'item_id':8153,
             'buy_price':111,
             'min_price':222,
             'normal_price':222,
             'max_price':222},

            {'name':'Briarthorn',
             'item_id':2450,
             'buy_price':111,
             'min_price':222,
             'normal_price':222,
             'max_price':222}
           ])
    

def example_export_output():
    EXPORT(export_example_data())
    
    
print(''' 
Usage:
  EXPORT(dataframe) --> expects df with specific columns
                          name         - required, string
                          item_id      - required, integer
                          buy_price    - required, integer in copper
                          min_price    - required, integer in copper 
                          normal_price - required, integer in copper
                          max_price    - required, integer in copper
                          restock      - optional, default 20
  example_export_input()  --> shows expected data schema
  example_export_output() --> shows output of example data
''') 

 
Usage:
  EXPORT(dataframe) --> expects df with specific columns
                          name         - required, string
                          item_id      - required, integer
                          buy_price    - required, integer in copper
                          min_price    - required, integer in copper 
                          normal_price - required, integer in copper
                          max_price    - required, integer in copper
                          restock      - optional, default 20
  example_export_input()  --> shows expected data schema
  example_export_output() --> shows output of example data

